### Read Smallcases

In [5]:
import re
import numpy as np
import pandas as pd
import requests

local_base_url = "http://localhost:2000"

In [6]:
# Connect to services
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)

from app.internal.firebase import db, bucket

In [9]:
ids = [
    {"id": "SCAW_0001", "name": "All Weather Investing"},
    {"id": "SCMO_0015", "name": "Safe Haven"},
    {"id": "SCMO_0026", "name": "Growth & Income"},
    {"id": "SCMO_0016", "name": "The Naked Trader"},
    {"id": "SCMO_0024", "name": "The PE List"},
    {"id": "SCMO_0013", "name": "Dividend Stars"},
    {"id": "SCMO_0014", "name": "Dividend Aristocrats"},
    {"id": "SCNM_0025", "name": "Electric Mobility"},
    {"id": "SCNM_0012", "name": "Rising Rural Demand"},
    {"id": "SCTR_0014", "name": "House of Tata"},
    {"id": "SCTR_0017", "name": "House of Murugappa"},
    {"id": "SCTR_0010", "name": "Realty Tracker"},
]
smallcase_id = ids[0]["id"]
smallcase_sheet_path = f"data/smallcases/{smallcase_id}_Timeline.xlsx"
smallcase_sheet = pd.ExcelFile(smallcase_sheet_path)

constituents = pd.read_excel(smallcase_sheet, sheet_name="Historical Constituents")
constituents.head()

,Date Range,Constituents,Weightage
0,2018-07-16 to 2018-07-16,Nippon India ETF Nifty 50 BeES,0.19
1,NaN,Nippon India ETF Nifty Next 50 Junior BeES,0.14
2,NaN,Nippon India ETF Gold BeES,0.41
3,NaN,Nippon India ETF Nifty 1D Rate Liquid BeES,0.25
4,2018-07-17 to 2018-09-27,Nippon India ETF Nifty 50 BeES,0.24


In [10]:
indexes = pd.read_excel(smallcase_sheet, sheet_name="Historical Index Values")
indexes.head()

,Date,All Weather Investing,NIFTY 100,Rebalance Occured
0,2018-07-17,100.00,100.00,True
1,2018-07-18,99.33,99.63,NaN
2,2018-07-19,99.29,99.37,NaN
3,2018-07-20,99.52,99.87,NaN
4,2018-07-23,100.22,100.68,NaN


### Fetch and POST Smallcase and Stats

In [285]:
smallcase_base_url = "https://api.smallcase.com"
resp = requests.get(f"{smallcase_base_url}/smallcases/smallcase?scid={smallcase_id}")

resp.status_code

200

In [286]:
smallcase = resp.json()["data"]
stats = smallcase["stats"]

def remove_p_tag(text: str):
    return re.sub(r"</?(p|ul|li)>", "", text)

methodologies = []
for m in smallcase["methodology"]:
    methodologies.append({
        "key": m["key"],
        "details": remove_p_tag(m["content"]),
    })

investment_strategies = []
for i in smallcase["info"]["investmentStrategy"]:
    investment_strategies.append(i["key"])

payload = {
    "id": smallcase_id,
    "name": smallcase["info"]["name"],
    "slug": smallcase["info"]["slug"],
    "description": smallcase["info"]["shortDescription"],
    "volatility": stats["ratios"]["riskLabel"],
    "popularity_rank": smallcase["flags"]["popular"]["rank"],
    "contains_etf": smallcase["flags"]["containsEtf"],
    "contains_stock": smallcase["flags"]["containsStock"],
    "constituent_count": smallcase["constituentsCount"],
    "growth_since_launch": {
        "cagr": stats["ratios"]["cagr"],
        "returns": stats["returns"]["sinceInception"],
        "duration": stats["ratios"]["cagrDuration"],
    },
    "benchmark": {
        "id": smallcase["benchmark"]["id"],
        "index": smallcase["benchmark"]["index"],
        "details": smallcase["benchmark"]["msg"],
    },
    "methodologies": methodologies,
    "launch_date": smallcase["info"]["uploaded"],
    "inception_date": smallcase["info"]["created"],
    "last_rebalance_date": smallcase["info"]["lastRebalanced"],
    "next_rebalance_date": smallcase["info"]["nextUpdate"],
    "investment_strategies": investment_strategies,
    "rebalance_frequency": smallcase["info"]["rebalanceSchedule"],
}

local_resp = requests.post(f"{local_base_url}/smallcases/", json=payload)
local_resp.status_code == 201

True

In [287]:
year = "2024"
month = "05"

payload = {
    "year": year,
    "month": month,
    "min_sip_amount": stats["minSipAmount"],
    "investor_count": stats["investorCount"],
    "subscriber_count": stats["subscriberCount"],
    
    "cagr": {
        "one_year": stats["ratios"]["cagr1y"],
        "three_year": stats["ratios"]["cagr3y"],
        "five_year": stats["ratios"]["cagr5y"],
    },
    "returns": {
        "monthly": stats["returns"]["monthly"],
        "quarterly": stats["returns"]["quarterly"],
        "half_year": stats["returns"]["halfyearly"],
        "one_year": stats["returns"]["yearly"],
        "three_year": stats["returns"]["threeYear"],
        "five_year": stats["returns"]["fiveYear"],
    },
    "ratios": {
        "dividend_yield": stats["ratios"]["divYield"],
        "dividend_yield_differential": stats["ratios"]["divYieldDifferential"],

        "risk": stats["ratios"]["risk"],
        "pe": stats["ratios"]["pe"],
        "pb": stats["ratios"]["pb"],
        "beta": stats["ratios"]["beta"],
        "sharpe": stats["ratios"]["sharpeRatio"],
    },
    "weightage": {
        "large_cap": stats["ratios"]["largeCapPercentage"],
        "mid_cap": stats["ratios"]["midCapPercentage"],
        "small_cap": stats["ratios"]["smallCapPercentage"],
        "market_category": stats["ratios"]["marketCapCategory"]
    }
}

local_resp = requests.post(f"{local_base_url}/smallcases/{smallcase_id}/statistics/", json=payload)
local_resp.status_code == 201

True

### Upload XLSX to bucket

In [288]:
blob = bucket.blob(f"smallcases/{smallcase_id}/timeline.xlsx")
blob.upload_from_filename(smallcase_sheet_path)

### Map and POST Smallcase Name to Upstox Instruments

In [289]:
postable_constituents = constituents["Constituents"].unique()
len(postable_constituents)

19

In [290]:
successful_constitents = []
failed_constitents = []

async def query_ticker_name(name: str):
    return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()

for c in postable_constituents:
    q = c.upper()
    docs = await query_ticker_name(q)
    if len(docs) == 0:
        failed_constitents.append(c)
    else:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": c
            })
            successful_constitents.append((c, e.id))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_57447/2676511566.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [291]:
len(successful_constitents), len(failed_constitents)

(4, 15)

In [292]:
# # All return upper

def replace_ltd(s):
    return s.upper().replace("LTD", "LIMITED")

# def replace_technologies(s):
#     return s.upper().replace("TECHNOLOGIES", "TECHNO.")

# def replace_company(s):
#     return s.upper().replace("COMPANY", "CO")

# def replace_corporation(s):
#     return s.upper().replace("CORPORATION", "CORP")

# Ltd -> Limited
for f in failed_constitents:
    q = replace_ltd(f)
    docs = await query_ticker_name(q)
    if len(docs) != 0:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": f
            })
            successful_constitents.append((f, e.id))
            failed_constitents.remove(f)

/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_57447/2676511566.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [293]:
len(successful_constitents), len(failed_constitents)

(8, 11)

In [294]:
nse = pd.read_csv("data/upstox/NSE.csv")
equities = nse.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [295]:
import difflib
df = equities.filter(['name', "exchange_token"])
tickers = pd.DataFrame() 
r = {}
for f in failed_constitents:
    df['similarity'] = df["name"].apply(lambda x: difflib.SequenceMatcher(None, x.lower(), f.lower()).ratio())
        
    top_match = df.sort_values(by='similarity', ascending=False).head(1)
    top_match['smallcase_name'] = f
        
    tickers = pd.concat([tickers, top_match], ignore_index=True)

tickers

,name,exchange_token,similarity,smallcase_name
0,DLF LIMITED,14732.0,0.777778,DLF Ltd
1,HOUSING DEV & INFRA LTD,17920.0,0.676923,Housing Development and Infrastructure Ltd
2,INDIABULLS REAL EST. LTD,14450.0,0.920000,Indiabulls Real Estate Ltd
3,THE PHOENIX MILLS LTD,14552.0,0.894737,Phoenix Mills Ltd
4,PRESTIGE ESTATE LTD,20302.0,0.791667,Prestige Estates Projects Ltd
5,UNITECH LIMITED,8893.0,0.846154,Unitech Ltd
6,BRIGADE ENTER. LTD,15184.0,0.829268,Brigade Enterprises Ltd
7,PURAVANKARA LIMITED,14926.0,0.882353,Puravankara Ltd
8,MAHINDRA LIFESPACE DEVLTD,8050.0,0.862069,Mahindra Lifespace Developers Ltd
9,KOLTE PATIL DEV. LTD.,15124.0,0.765957,Kolte-Patil Developers Ltd


In [296]:
# incorrect = [0, 1, 7, 10, 24, 26]
# corrections = {
#     "7": "11782",
#     "26": "15034"
# }

# cannot_correct = len(incorrect) - len(corrections)
# for i in incorrect:
#     n = tickers.iloc[i]["smallcase_name"]
#     correct_token = corrections.get(str(i), None)
#     if correct_token:
#         correct = equities.query(f"exchange_token == {correct_token}").iloc[0]
#         tickers.loc[i, ['exchange_token', 'name', 'similarity']] = [correct_token, correct['name'], 1.0]

# for i in incorrect:
#     correct_token = corrections.get(str(i), None)
#     if not correct_token:
#         tickers.drop(i, inplace=True)
# cannot_correct, tickers.shape

2, 15 for Safe Haven do not exist (Polaris Consulting & Monsanto were merged w other companies).
Trade-off -> 

We cannot fetch market data for these stocks, so we do not add them to the equities list.
When rebalancing for Kelly, we will ignore these stocks, maintaining their original weightage and assess the implications.

In [297]:
payload = tickers[['exchange_token', 'smallcase_name']].to_dict(orient='records')
for p in payload:
    id = str(int(p['exchange_token']))
    await db.document(f"tickers/{id}").update({"smallcase_name": p["smallcase_name"]})
    successful_constitents.append((p["smallcase_name"], id))

In [298]:

len(successful_constitents), len(successful_constitents) == (len(postable_constituents) - 0)

(19, True)

All Weather = 5

Safe Haven = 73

Growth & Income = 102 (~10 corrections)

The Naked Trader = 115 (~12 corrections)

The PE List = 37 (7 corrections, 3 possible)

Dividend Stars = 38

Dividend Aristocrats = 31 (3 corrections, 2 possible)

Electric Mobility = 29 (1 correction)

Rising Rural Demand = 52 (6 corrections, 4 possible)

The House of Tata = 11

The House of Murugappa = 5

Realty Tracker = 19


**517 total, 347 unique (after querying db)**

### POST Smallcase Constituents

In [115]:
def get_constituent_payload(name: str, weight: float):
    return {
        "smallcase_name": name,
        "original_weightage": weight
    }

In [120]:
constituents_group = constituents.fillna(method='ffill').groupby("Date Range").apply(lambda x:[get_constituent_payload(row["Constituents"], row["Weightage"]) for i, row in x.iterrows()]).reset_index(name='Constituents')
constituents_group.head()

/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_53960/3640253601.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  constituents_group = constituents.fillna(method='ffill').groupby("Date Range").apply(lambda x:[get_constituent_payload(row["Constituents"], row["Weightage"]) for i, row in x.iterrows()]).reset_index(name='Constituents')
/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_53960/3640253601.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  constituents_group = constituents.fillna(method='ffill').groupby("Date Range").apply(lambda x:[get_constituent_payload(row["Constituents"

,Date Range,Constituents
0,2016-04-04 to 2016-05-31,"[{'smallcase_name': 'Delta Corp Ltd', 'origina..."
1,2016-06-01 to 2016-08-31,"[{'smallcase_name': 'Delta Corp Ltd', 'origina..."
2,2016-09-01 to 2016-11-30,"[{'smallcase_name': 'Delta Corp Ltd', 'origina..."
3,2016-12-01 to 2017-02-28,"[{'smallcase_name': 'DLF Ltd', 'original_weigh..."
4,2017-03-01 to 2017-05-31,"[{'smallcase_name': 'DLF Ltd', 'original_weigh..."


In [121]:
s, f = 0, 0
failed = []
for i, row in constituents_group.iterrows():
    start_date, end_date = row["Date Range"].split(" to ")
    payload = {
        "start_date": start_date,
        "end_date": end_date,
        "constituents": row["Constituents"]
    }

    resp = requests.post(f"{local_base_url}/smallcases/{smallcase_id}/constituents", json=payload)
    if resp.status_code == 201:
        s += 1
    else:
        f += 1
        print(resp.json())
        failed.append(row)

print(f"Success: {s}, Failed: {f}")

Success: 33, Failed: 0


Smallcase - Unique date ranges successful, failed

All Weather - 25, 0

Safe Haven - 33, 0

Growth & Income - 31, 0

The Naked Trader - 32, 0

The PE List - 9, 0

Dividend Stars - 12, 0

Dividend Aristorcrats - 12, 0

Electric Mobility - 27, 0

Rising Rural Demand - 33, 0

House of Tata - 14, 0

House of Murugappa - 8, 0

Realty Tracker - 33, 0

### POST Smallcase Indexes

In [39]:
smallcase_col = indexes.columns[1]
benchmark_col = indexes.columns[2]

# Wrangle indexes
indexes['Date'] = pd.to_datetime(indexes['Date'])
indexes.fillna(False)
indexes.replace('-', pd.NA, inplace=True)
indexes.fillna(method='ffill', inplace=True)
monthly = indexes.groupby([indexes['Date'].dt.year, indexes['Date'].dt.month])

monthly.replace('-', pd.NA, inplace=True)
monthly.fillna(method='ffill', inplace=True)

monthly.head()

# s, f = 0, 0
# failed = []
# for (y, m), month in monthly:
#     if y == 2020 and m == 2:
#         start = month['Date'].min().strftime('%Y-%m-%d')
#         end = month['Date'].max().strftime('%Y-%m-%d')
#         print(start)
#         # payload = {
#         #     "indexes": {},
#         #     "start_date": start,
#         #     "end_date": end,
#         # }
#         # for index, row in month.iterrows():
#         #     date_str = row['Date'].strftime('%Y-%m-%d')
#         #     payload["indexes"][date_str] = {
#         #         "smallcase": row[smallcase_col],
#         #         "benchmark": row[benchmark_col],
#         #         "rebalance_occured": row['Rebalance Occured']
#         #     }
#         # resp = requests.post(f"{local_base_url}/smallcases/{smallcase_id}/indexes", json=payload, params={"date": start})
#         # if resp.status_code == 201:
#         #     s += 1
#         # else:
#         #     f += 1
#         #     print(resp.json())
#         #     failed.append((y, m))

# print(f"Success: {s}, Failed: {f}")

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_54915/1818771318.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  monthly = indexes.fillna(False).groupby([indexes['Date'].dt.year, indexes['Date'].dt.month])


AttributeError: 'DataFrameGroupBy' object has no attribute 'replace'